In [ ]:
import tensorflow as tf
import pathlib
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras.applications import EfficientNetB4, EfficientNetB5
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from sklearn.model_selection import KFold


from zipfile import ZipFile

with ZipFile('/content/New_Dataset.zip', 'r') as zipObj:
    zipObj.extractall('IMAGES')

data_dir = pathlib.Path('/content/IMAGES/New_Dataset')

train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(224, 224),
    batch_size=8
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(224, 224),
    batch_size=8
)

normalization_layer = Rescaling(1./255)
train_ds_normalized = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds_normalized = val_ds.map(lambda x, y: (normalization_layer(x), y))

num_classes = 2
def one_hot_encoding(x, y):
    return x, tf.one_hot(y, depth=num_classes)

train_ds_encoded = train_ds_normalized.map(one_hot_encoding)
val_ds_encoded = val_ds_normalized.map(one_hot_encoding)

train_ds_encoded = train_ds_encoded.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
val_ds_encoded = val_ds_encoded.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)


base_model_b4 = EfficientNetB5(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x_b4 = base_model_b4.output
x_b4 = GlobalAveragePooling2D()(x_b4)
x_b4 = Dense(512, activation='relu')(x_b4)
x_b4 = Dropout(0.20)(x_b4)
x_b4 = Dense(256, activation='relu')(x_b4)
output_b4 = Dense(num_classes, activation='softmax')(x_b4)
model_efficientnet_b4 = Model(inputs=base_model_b4.input, outputs=output_b4)
model_efficientnet_b4.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


num_folds = 5
fold_accuracies = []


train_images, train_labels = tf.concat([x for x, y in train_ds_encoded], axis=0).numpy(), tf.concat([y for x, y in train_ds_encoded], axis=0).numpy()

kf = KFold(n_splits=num_folds, shuffle=True, random_state=123)

for fold, (train_index, val_index) in enumerate(kf.split(train_images)):
    print(f"Training Fold {fold+1}/{num_folds}")


    x_train, x_val = train_images[train_index], train_images[val_index]
    y_train, y_val = train_labels[train_index], train_labels[val_index]


    train_ds_fold = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(1000).batch(16)
    val_ds_fold = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(16)


    model_efficientnet_b4.fit(
        train_ds_fold,
        epochs=25,
        verbose=1,
        validation_data=val_ds_fold
    )


    _, accuracy = model_efficientnet_b4.evaluate(val_ds_fold)
    fold_accuracies.append(accuracy)


average_accuracy = sum(fold_accuracies) / len(fold_accuracies)
print("Average Accuracy: {:.2f}%".format(average_accuracy * 100))
